In [ ]:
import pandas as pd

dados_treino = pd.read_csv('https://caelum-online-public.s3.amazonaws.com/1638-word-embedding/treino.csv')
dados_teste = pd.read_csv('https://caelum-online-public.s3.amazonaws.com/1638-word-embedding/teste.csv')


In [ ]:
dados_treino["title"].head()

0    Após polêmica, Marine Le Pen diz que abomina n...
1    Macron e Le Pen vão ao 2º turno na França, em ...
2    Apesar de larga vitória nas legislativas, Macr...
3    Governo antecipa balanço, e Alckmin anuncia qu...
4    Após queda em maio, a atividade econômica sobe...
Name: title, dtype: object

In [ ]:
!python -m spacy download pt_core_news_sm

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-01-05 19:03:05.548667: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 63.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


In [ ]:
import spacy
nlp = spacy.load("pt_core_news_sm")

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
nlp

In [ ]:
texto = "Fortaleza é uma cidade incrível"

doc = nlp(texto)

In [ ]:
type(doc)

spacy.tokens.doc.Doc

In [ ]:
doc[0]

Fortaleza

In [ ]:
type(doc[0])

spacy.tokens.token.Token

In [ ]:
doc.ents

(Fortaleza,)

In [ ]:
doc[1].is_stop

True

In [ ]:
textos_para_tratamento = (titulos.lower() for titulos in dados_treino["title"])

In [ ]:
def trata_textos(doc):
  tokens_validos = []

  for token in doc:
    eh_valido = not token.is_stop and token.is_alpha
    if eh_valido:
      tokens_validos.append(token.text)

  if len(tokens_validos) > 2:
    return " ".join(tokens_validos)

In [ ]:
textos_tratados = [trata_textos(doc) for doc in nlp.pipe(textos_para_tratamento, batch_size=1000, n_process=-1)]

In [ ]:
titulos_tratados = pd.DataFrame({"titulo": textos_tratados})
titulos_tratados.head()

,titulo
0,polêmica marine le pen abomina negacionistas h...
1,macron le pen turno frança revés siglas tradic...
2,apesar larga vitória legislativas macron terá ...
3,governo antecipa balanço alckmin anuncia queda...
4,queda maio atividade econômica sobe junho bc


In [ ]:
from gensim.models import Word2Vec

w2v_modelo = Word2Vec(sg = 0, 
                      window = 2,
                      size = 300,
                      min_count = 5,
                      alpha = 0.03,
                      min_alpha = 0.007)

In [ ]:
w2v_modelo

In [ ]:
print(len(titulos_tratados))

titulos_tratados = titulos_tratados.dropna().drop_duplicates()

print(len(titulos_tratados))

90000
84466


In [ ]:
lista_lista_tokens = [titulo.split(" ") for titulo in titulos_tratados.titulo]

In [ ]:
w2v_modelo.build_vocab(lista_lista_tokens)

In [ ]:
import logging

logging.basicConfig(format="%(asctime)s : %(levelname)s : %(message)s", level=logging.INFO)

w2v_modelo = Word2Vec(sg = 0, 
                      window = 2,
                      size = 300,
                      min_count = 5,
                      alpha = 0.03,
                      min_alpha = 0.007)

w2v_modelo.build_vocab(lista_lista_tokens, progress_per=5000)

In [ ]:
dir(w2v_modelo)

['__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_adapt_by_suffix',
 '_check_input_data_sanity',
 '_check_training_sanity',
 '_clear_post_train',
 '_do_train_epoch',
 '_do_train_job',
 '_get_job_params',
 '_get_thread_working_mem',
 '_job_producer',
 '_load_specials',
 '_log_epoch_end',
 '_log_epoch_progress',
 '_log_progress',
 '_log_train_end',
 '_minimize_model',
 '_raw_word_count',
 '_save_specials',
 '_set_train_params',
 '_smart_save',
 '_train_epoch',
 '_train_epoch_corpusfile',
 '_update_job_params',
 '_worker_loop',
 '_worker_loop_corpusfile',
 'accuracy',
 'alpha',
 'batch_words',
 'build_vocab',
 'build_vocab_from_freq',
 'ca

In [ ]:
w2v_modelo.corpus_count

84466

In [ ]:
w2v_modelo.train(lista_lista_tokens, 
                 total_examples=w2v_modelo.corpus_count,
                 epochs = 30)

(14584330, 16207260)

In [ ]:
w2v_modelo.wv.most_similar("google")

[('apple', 0.6115872859954834),
 ('facebook', 0.522089958190918),
 ('airbnb', 0.5207315683364868),
 ('amazon', 0.5009369254112244),
 ('sony', 0.47806933522224426),
 ('volkswagen', 0.4672483205795288),
 ('unilever', 0.4637262523174286),
 ('buffett', 0.46024811267852783),
 ('uber', 0.4589741826057434),
 ('software', 0.4583141803741455)]

In [ ]:
w2v_modelo.wv.most_similar("microsoft")

[('amazon', 0.6006731986999512),
 ('sky', 0.5587750673294067),
 ('unilever', 0.5446935892105103),
 ('tesla', 0.5366277694702148),
 ('sony', 0.5318712592124939),
 ('buffett', 0.5303621888160706),
 ('walmart', 0.5250080227851868),
 ('chrysler', 0.5197633504867554),
 ('viajante', 0.5180898904800415),
 ('editora', 0.5139142274856567)]

In [ ]:
w2v_modelo.wv.most_similar("barcelona ")

[('bayern', 0.5799539089202881),
 ('psg', 0.5772843956947327),
 ('barça', 0.5730800628662109),
 ('chelsea', 0.5724823474884033),
 ('united', 0.5669465065002441),
 ('juventus', 0.5612977147102356),
 ('madrid', 0.5551157593727112),
 ('lazio', 0.5544372797012329),
 ('munique', 0.5482319593429565),
 ('betis', 0.5348033905029297)]

In [ ]:
w2v_modelo.wv.most_similar("messi")

[('suárez', 0.5919152498245239),
 ('cristiano', 0.5519205927848816),
 ('cavani', 0.5405541658401489),
 ('benzema', 0.5054856538772583),
 ('barcelona', 0.5049266219139099),
 ('chuteiras', 0.5047380924224854),
 ('psg', 0.49956467747688293),
 ('ronaldo', 0.4960896074771881),
 ('barça', 0.4904443323612213),
 ('neymar', 0.4815254807472229)]

In [ ]:
w2v_modelo.wv.most_similar("gm")

[('chrysler', 0.6860204935073853),
 ('embraer', 0.6752134561538696),
 ('volks', 0.6440712809562683),
 ('honda', 0.6351333260536194),
 ('braskem', 0.6199281215667725),
 ('volkswagen', 0.6039223670959473),
 ('fiat', 0.6014561653137207),
 ('inbev', 0.5968788862228394),
 ('mitsubishi', 0.5894317626953125),
 ('toyota', 0.5879557728767395)]

In [ ]:
#treinamento do modelo skip-gram
w2v_modelo_sg = Word2Vec(sg = 1, 
                      window = 5,
                      size = 300,
                      min_count = 5,
                      alpha = 0.03,
                      min_alpha = 0.007)

w2v_modelo_sg.build_vocab(lista_lista_tokens, progress_per=5000)

w2v_modelo_sg.train(lista_lista_tokens, 
                 total_examples=w2v_modelo_sg.corpus_count,
                 epochs = 30)

(14584359, 16207260)

In [ ]:
w2v_modelo_sg.wv.most_similar("google")

[('reguladores', 0.41288667917251587),
 ('apple', 0.3982684314250946),
 ('waze', 0.38477692008018494),
 ('android', 0.38451144099235535),
 ('facebook', 0.37902653217315674),
 ('buffett', 0.37052130699157715),
 ('anunciantes', 0.3703813850879669),
 ('patentes', 0.3594432473182678),
 ('verizon', 0.35806918144226074),
 ('concorda', 0.35201844573020935)]

In [ ]:
w2v_modelo.wv.most_similar("google")

[('apple', 0.6115872859954834),
 ('facebook', 0.522089958190918),
 ('airbnb', 0.5207315683364868),
 ('amazon', 0.5009369254112244),
 ('sony', 0.47806933522224426),
 ('volkswagen', 0.4672483205795288),
 ('unilever', 0.4637262523174286),
 ('buffett', 0.46024811267852783),
 ('uber', 0.4589741826057434),
 ('software', 0.4583141803741455)]

In [ ]:
w2v_modelo_sg.wv.most_similar("gm")

[('metalúrgicos', 0.5765355825424194),
 ('motors', 0.5387244820594788),
 ('montadora', 0.5017637014389038),
 ('audi', 0.4869466722011566),
 ('honda', 0.48437321186065674),
 ('cubatão', 0.47674107551574707),
 ('airbags', 0.47570183873176575),
 ('compartilhamento', 0.4742475152015686),
 ('bmw', 0.4637686312198639),
 ('coletivas', 0.4627327024936676)]

In [ ]:
w2v_modelo.wv.most_similar("gm")

[('chrysler', 0.6860204935073853),
 ('embraer', 0.6752134561538696),
 ('volks', 0.6440712809562683),
 ('honda', 0.6351333260536194),
 ('braskem', 0.6199281215667725),
 ('volkswagen', 0.6039223670959473),
 ('fiat', 0.6014561653137207),
 ('inbev', 0.5968788862228394),
 ('mitsubishi', 0.5894317626953125),
 ('toyota', 0.5879557728767395)]

In [ ]:
w2v_modelo.wv.save_word2vec_format("/content/drive/MyDrive/Word2Vec/modelo_cbow.txt", binary=False)
w2v_modelo_sg.wv.save_word2vec_format("/content/drive/MyDrive/Word2Vec/modelo_skip.txt", binary=False)